<h1><center> Tutorial 2: Running Experiments </center></h1>

---

This tutorial teaches you how to use pyGOLD's `run_solvers()` function to run optimization experiments. You'll learn about about experiment options and the available benchmark problems.

In [1]:
import pygold
from pygold.optimizer import BaseOptimizer, OptimizationResult
from scipy.optimize import dual_annealing, basinhopping

## Part 1: The run_solvers() Function
---

The `run_solvers()` function is the core of pyGOLD's benchmarking functionality. It runs optimization algorithms on benchmark problems and records performance data.

### Function Signature:
```python
run_solvers(solvers, problems, test_dimensions=[2, 4, 5, 8, 10, 12], n_iters=5, output_folder=None, track_energy=True, verbose=False)
```

### Parameter Breakdown:
- solvers (Required): List of algorithms to solve. Each algorithm must be derived from the `BaseOptimizer` class.
- problems (Required): List of benchmark algorithm classes to test the algorithms on.
- test_dimensions (Optional, Default 2, 4, 5, 8, 10, 12): Specifies which dimensions to test for variable-dimension (nD) problems.
  - nD benchmark problems will be tested in each dimension.
  - Ignored for fixed-dimension problems.
- n_iters (Optional, Default 5):  Number of independent runs per algorithm per problem.
  - If an algorithm is deterministic and requires no initial points, then it is only run once. (Repeated runs would be identical.)
  - All other algorithms are run n_iters times.
- output_folder (Optional, default "output_data"): Where to save performance data. Creates:
  -  `.info` files with problem and algorithm metadata
  -  `.dat`, `.tdat`, `.mdat` files with aw function evaluation data
  -  `.csv` file with power consumption measurements (if available)
- track_energy (Optional, default True): Whether to track energy consumption data
  - Requires the CodeCarbon library
  - If the CodeCarbon library is not installed, energy consumption data will not be recorded but benchmarking will be unaffected.
- verbose (Optional, default False): Whether to print progress information


## Part 2: Available Problems
---

Now let's explore the types of problems available in pyGOLD.

### Standard Problems:

pyGOLD offers many classical test functions for benchmarking, including every optimization problem from the Virtual Library of Simulation Experiments. These problems are organized by tags and can be accessed using the `get_standard_problems()` function. Supported tags include: 
- `Unconstrained` / `Constrained`: Whether the function has constraints
- `Multimodal` / `Unimodal`: Number of local/global minima
- `Continuous` / `Discontinuous`: Whether the function is continuous - functions with sharp ridges or drops are classified as discontinuous
- `Differentiable` / `Non_differentiable`: Whether the function is differentiable
- `Separable` / `Non_separable`: Whether the function can be separated into independent subproblems
- `1D`, `2D`, `nD`: Dimensionality of the function

You can use these tags individually or combine them. For example:

In [2]:
# Get 2D problems
problems = pygold.get_standard_problems(["2D", "Unconstrained"])
print(f"Found {len(problems)} 2D, unconstrained problems:")
for i, problem in enumerate(problems[:10], 1):  # Show first 10
    print(f"{i:2d}. {problem.__name__:20}")

if len(problems) > 10:
    print(f"   ... and {len(problems) - 10} more problems\n\n")


# Get nD, unconstrained problems
problems = pygold.get_standard_problems(["nD", "Differentiable", "Unconstrained"])
print(f"Found {len(problems)} nD, differentiable, unconstrained problems:")
for i, problem in enumerate(problems[:10], 1):
    print(f"{i:2d}. {problem.__name__:20}")
if len(problems) > 10:
    print(f"   ... and {len(problems) - 10} more problems")

Found 23 2D, unconstrained problems:
 1. Dejong5             
 2. DropWave            
 3. Levy13              
 4. Branin              
 5. Bohachevsky2        
 6. Camel6              
 7. CrossInTray         
 8. Bohachevsky1        
 9. Camel3              
10. GoldsteinPrice      
   ... and 13 more problems


Found 15 nD, differentiable, unconstrained problems:
 1. SumSq               
 2. Schwefel            
 3. Michalewicz         
 4. Sphere              
 5. StyblinskiTang      
 6. Perm0               
 7. Powell              
 8. Rosenbrock          
 9. Ackley              
10. Trid                
   ... and 5 more problems


### FLORIS Problems

In addition to the standard problems, pyGOLD offers wind farm optimization problems using FLORIS. These are available in the `pygold.problems.floris` module. Currently available are:
- TurbineLayout: Control the (x,y) locations of turbines in a wind farm to maximize Annual Energy Production.
- TurbineLayoutYaw: Control the (x,y) locations and the turbine offset angle from the wind.

Stochastic versions of these problems are also available. These are made by adding noise to the sampling points of the wind condition PDF.

For more on the FLORIS problems, see ``ex2_floris.ipynb``.

## Part 3: Example
---

Let's put it all together with an example:

In [3]:
# 1. Define algorithms
class DualAnnealing(BaseOptimizer):
    deterministic = False
    n_points = 0
    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        result = dual_annealing(func, bounds, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

class BasinHopping(BaseOptimizer):
    deterministic = False
    n_points = 1
    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        result = basinhopping(func, x0, minimizer_kwargs={'bounds': bounds}, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

my_solvers = [DualAnnealing(), BasinHopping()]

# 2. Select problems
my_problems = pygold.get_standard_problems(["2D", "Unconstrained"])[:3]

# 3. Run experiment with all parameters
pygold.run_solvers(
    solvers=my_solvers,                    # Algorithm instances
    problems=my_problems,                  # Problem classes
    test_dimensions=[2],                   # Test in 2D - note that since we're only using 2D problems, this is redundant
    n_iters=5,                             # 5 independent runs
    output_folder="tutorial2_data",        # Results folder
    verbose=False,                         # Show progress
    track_energy=False                     # No energy tracking
)

print("Experiment complete.\nResults saved in: tutorial2_data/")

Experiment complete.
Results saved in: tutorial2_data/


## Summary

You now understand the parameters of run_solvers(), the problems available within pyGOLD, and how to access them.

Next: **Tutorial 3** will show you how to use the postprocessing tools within pyGOLD to analyze and visualize the results from your experiments.